### imports 

In [1]:
import numpy as np
import pygame
import random

pygame 2.4.0 (SDL 2.26.4, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


### constants

In [2]:
# Game Constants
WINDOW_WIDTH = 560
WINDOW_HEIGHT = 530
GRID_SIZE = 80
PADDING = 10
BOARD_WIDTH = 7
BOARD_HEIGHT = 6
WINDOW_LENGTH = 4

EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2


# Colors
BACKGROUND_COLOR = (0, 0, 255)
EMPTY_COLOR = (0, 0, 0)
PLAYER_COLOR = (255, 0, 0)
AI_COLOR = (255, 200, 0)

### Connect4 model 

In [3]:
class ConnectFourModel:
    def __init__(self):
        self.board = np.zeros((BOARD_HEIGHT, BOARD_WIDTH), dtype=np.int8)
        self.current_player = 1

    def get_board(self):
        return self.board

    def get_current_player(self):
        return self.current_player

    def get_valid_moves(self):
        valid_moves = []
        for col in range(BOARD_WIDTH):
            if self.board[BOARD_HEIGHT - 1][col] == 0:
                valid_moves.append(col)
        return valid_moves

    def make_move(self, column):
        for row in range(BOARD_HEIGHT):
            if self.board[row][column] == 0:
                self.board[row][column] = self.current_player
                self.current_player = 3 - self.current_player
                return True
        return False

    def undo_move(self, column):
        for row in range(BOARD_HEIGHT - 1, -1, -1):
            if self.board[row][column] != 0:
                self.board[row][column] = 0
                self.current_player = 3 - self.current_player
                return

    def is_terminal(self):
        return len(self.get_valid_moves()) == 0 or self.check_winner() is not None
    
    def check_winner(self):
        
            lines = self.get_lines(PLAYER_PIECE)
            for line in lines:
                if self.is_winning_line(line,PLAYER_PIECE):
                    return PLAYER_PIECE
                
            lines = self.get_lines(AI_PIECE)
            for line in lines:
                if self.is_winning_line(line,AI_PIECE):
                    return AI_PIECE
            return None
    

    def get_lines(self, player):
        lines = []

        # Horizontal lines
        for row in range(BOARD_HEIGHT):
            for col in range(BOARD_WIDTH - 3):
                lines.append(self.board[row, col:col + 4])

        # Vertical lines
        for col in range(BOARD_WIDTH):
            for row in range(BOARD_HEIGHT - 3):
                lines.append(self.board[row:row + 4, col])

        # Diagonal lines
        for row in range(BOARD_HEIGHT - 3):
            for col in range(BOARD_WIDTH - 3):
                lines.append(self.board[row:row + 4, col:col + 4].diagonal())

            for col in range(3, BOARD_WIDTH):
                lines.append(np.fliplr(self.board[row:row + 4, col - 3:col + 1]).diagonal())

        return lines

    def is_winning_line(self, line, PIECE):
        return np.array_equal(line, [PIECE]*4) 
    

### Minimax Agent


In [4]:
class MinimaxAgent:
    def __init__(self, max_depth=4):
        self.max_depth = max_depth

    def choose_move(self, model, enable_pruning=True):
        best_move, best_score = self.minimax(model, self.max_depth, float('-inf'), float('inf'), True, enable_pruning)
        return best_move

    def minimax(self, model, depth, alpha, beta, maximizing_player, enable_pruning):
        if depth == 0 or model.is_terminal():
            return None, self.heuristic(model)

        if maximizing_player:
            max_score = float('-inf')
            best_move = None
            valid_moves = model.get_valid_moves()

            for move in valid_moves:
                model.make_move(move)
                _, score = self.minimax(model, depth - 1, alpha, beta, False, enable_pruning)
                model.undo_move(move)

                if score > max_score:
                    max_score = score
                    best_move = move

                if enable_pruning:
                    alpha = max(alpha, score)
                    if beta <= alpha:
                        break

            return best_move, max_score
        else:
            min_score = float('inf')
            best_move = None
            valid_moves = model.get_valid_moves()

            for move in valid_moves:
                model.make_move(move)
                _, score = self.minimax(model, depth - 1, alpha, beta, True, enable_pruning)
                model.undo_move(move)

                if score < min_score:
                    min_score = score
                    best_move = move

                if enable_pruning:
                    beta = min(beta, score)
                    if enable_pruning and beta <= alpha:
                        break

            return best_move, min_score
        

    def heuristic(self, model):
        board = model.get_board()
        return self.score_position(board, AI_PIECE) - self.score_position(board, PLAYER_PIECE)
    
    
    
    def evaluate_window(self, window, piece):
        score = 0
        opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(EMPTY) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(EMPTY) == 2:
            score += 2

        if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
            score -= 4

        return score
    

    
    def score_position(self, board, piece):
        score = 0

        # Score center column
        center_array = [int(i) for i in list(board[:, BOARD_WIDTH // 2])]
        center_count = center_array.count(piece)
        score += center_count * 3

        # Score Horizontal
        for r in range(BOARD_HEIGHT):
            row_array = [int(i) for i in list(board[r, :])]
            for c in range(BOARD_WIDTH - 3):
                window = row_array[c:c + WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        # Score Vertical
        for c in range(BOARD_WIDTH):
            col_array = [int(i) for i in list(board[:, c])]
            for r in range(BOARD_HEIGHT - 3):
                window = col_array[r:r + WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        # Score positive sloped diagonal
        for r in range(BOARD_HEIGHT - 3):
            for c in range(BOARD_WIDTH - 3):
                window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)

        for r in range(BOARD_HEIGHT - 3):
            for c in range(BOARD_WIDTH - 3):
                window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)

        return score



### Connect4 View

In [5]:
class ConnectFourView:
    def __init__(self):
        pygame.init()
        self.clock = pygame.time.Clock()
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Connect Four")
        self.font = pygame.font.SysFont(None, 48)

    def set_model(self, model):
        self.model = model

    def draw_board(self):
        self.screen.fill(BACKGROUND_COLOR)

        board = self.model.get_board()
        for row in range(BOARD_HEIGHT):
            for col in range(BOARD_WIDTH):
                color = EMPTY_COLOR
                if board[row][col] == 1:
                    color = PLAYER_COLOR
                elif board[row][col] == 2:
                    color = AI_COLOR

                pygame.draw.circle(self.screen, color, (col * GRID_SIZE + GRID_SIZE // 2, (BOARD_HEIGHT - row) * GRID_SIZE), GRID_SIZE // 2 - PADDING)

        pygame.display.flip()

    def display_winner(self, winner):
        if winner is None:
            text = self.font.render("It's a tie!", True, (255, 255, 255))
        else:
            text = self.font.render(f"Player {winner} wins!", True, (255, 255, 255))

        text_rect = text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        self.screen.blit(text, text_rect)
        pygame.display.flip()

    def get_user_move(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    x, _ = pygame.mouse.get_pos()
                    column = x // GRID_SIZE
                    if column in self.model.get_valid_moves():
                        return column

            self.clock.tick(60)
 

### Connect4 Game

In [6]:
class ConnectFourGame:
    def __init__(self):
        self.model = ConnectFourModel()
        self.view = ConnectFourView()
        self.agent = MinimaxAgent()
        self.game_over = False

    def run(self):
        self.view.set_model(self.model)

        while not self.game_over:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.game_over = True

            if not self.model.is_terminal():
                if self.model.get_current_player() == PLAYER_PIECE:
                    column = self.view.get_user_move()
                    self.make_move(column)
                else:
                    column = self.agent.choose_move(self.model)
                    self.make_move(column)

            self.view.draw_board()
            self.check_game_over()

    def make_move(self, column):
        if self.model.make_move(column):
            self.check_game_over()

    def check_game_over(self):
        winner = self.model.check_winner()
        if winner is not None or len(self.model.get_valid_moves()) == 0:
            self.game_over = True
            self.view.display_winner(winner)


##  Running a game

In [7]:
game = ConnectFourGame()
game.run()